#Εισαγωγικα
 Συνεργάτες :
 
  Θεοδότη Στόικου 03117085

  Δημήτρης Κουνούδης 03117169

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!chmod +x install_openfst.sh
!./install_openfst.sh &> /dev/null

In [ ]:
!pip install numpy
!pip install nltk
!pip install tqdm==4.48.2
!pip install contractions==0.0.25
!pip install gensim==3.8.3
!pip install scikit-learn==0.23.2
!pip install matplotlib==3.2.2
!pip install plotly==4.10.0
!sudo apt install python-pydot python-pydot-ng graphviz
#cd /content

#Βημα 1
1)a)Τα βήματα της προεπεξεργασίας του κειμένου λαμβάνουν χωρα στη συνάρτηση preprocess και συνεπώς και στις tokenize , clean text αφού έκαστη είναι εμφωλευμένη στην άλλη . Υπάρχει σχολιασμός στο κομμάτι του κώδικα πάνω στην ακριβή λειτουργία του καθενός. Αν θέλαμε να διατηρήσουμε τα σημεία στοίξης όπως την απόστροφο μπορούμε να αφαιρέσουμε τη συνάρτηση contractions.fix οπότε να διατηρήσουμε τη συμπτυγμένη μορφή των λέξεων και συνεπώς τις αποστρόφους .Επίσης θα μπορούσαμε να αφαιρέσουμε την εντολή   s = re.sub(r"[^a-z\s]", " ", s) που κρατάει μόνο τα lower case γράμματα ώστε να υιοθετήσουμε ενα λιγότερo "επιθετικό" preprocessing. Ετσι μπορούμε να διατηρήσουμε ένα πιο κατανοητό και αναλλοίωτο νόημα στο κείμενό μας.
b)Προσθέτοντας περισσότερα βιβλία θα μπορούσαμε να περιλάβουμε διαφορετικά 

In [ ]:
import re
import sys

import contractions
import nltk


def download_corpus(corpus="gutenberg"):
    """Download Project Gutenberg corpus, consisting of 18 classic books
    Book list:
       ['austen-emma.txt',
        'austen-persuasion.txt',
        'austen-sense.txt',
        'bible-kjv.txt',
        'blake-poems.txt',
        'bryant-stories.txt',
        'burgess-busterbrown.txt',
        'carroll-alice.txt',
        'chesterton-ball.txt',
        'chesterton-brown.txt',
        'chesterton-thursday.txt',
        'edgeworth-parents.txt',
        'melville-moby_dick.txt',
        'milton-paradise.txt',
        'shakespeare-caesar.txt',
        'shakespeare-hamlet.txt',
        'shakespeare-macbeth.txt',
        'whitman-leaves.txt']
    """
    nltk.download(corpus)
    raw = nltk.corpus.__getattr__(corpus).raw()

    return raw


def identity_preprocess(s):
    return s


def clean_text(s):
    s = s.strip()  # strip leading / trailing spaces
    s = s.lower()  # convert to lowercase
    s = contractions.fix(s)  # e.g. don't -> do not, you're -> you are
    s = re.sub("\s+", " ", s)  # strip multiple whitespace
    s = re.sub(r"[^a-z\s]", " ", s)  # keep only lowercase letters and spaces

    return s


def tokenize(s):
    tokenized = [w for w in s.split(" ") if len(w) > 0]  # Ignore empty string

    return tokenized


def preprocess(s):
    return tokenize(clean_text(s))


def process_file(corpus, preprocess=identity_preprocess):
    lines = [preprocess(ln) for ln in corpus.split("\n")]
    lines = [ln for ln in lines if len(ln) > 0]  # Ignore empty lines

    return lines


CORPUS = "gutenberg" 
raw_corpus = download_corpus(corpus=CORPUS)
preprocessed = process_file(raw_corpus, preprocess=preprocess)

#print(len(preprocessed))
# print(type(preprocessed))
#print(preprocessed[0:3])
# for words in preprocessed:
#     sys.stdout.write(" ".join(words))
#     sys.stdout.write("\n")
# print("done")
# print(preprocessed)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


#Βημα 2
  2) a)Δημιουργούμε το λεξικό που χρειάζεται τύπου dictionary

In [ ]:
lexicon = {}
for line in preprocessed:
  for word in line:
    if word in lexicon:
      lexicon[word] += 1 #if the word exist , increase by one its value
    else:
      lexicon[word] = 1 #if it doesn't exist initialize its value = 1
# print(word)

NameError: ignored

b)Εδώ φιλτράρουμε τις λέξεις που εμφανίζονται λιγότερο από πέντε φορές προσθέτοντας τις σε μια λίστα με τις λέξεις που πρέπει να διαγραφούν κι έπειτα τις διαγράφουμε.Αυτό το κάνουμε γιατι εφ'όσον το μοντέλο μας θα χρησιμοποιηθεί για στατιστικά στοιχεία δεν θέλουμε να κρατάμε λέξεις που εμφανίζονται πολυ λίγες φορές.



In [ ]:
todelete = []
for word in lexicon:
  if lexicon[word] < 5:
    todelete.append(word) #if its value is less than 5 add it to a list named todelete
    # print(word)

for word in todelete:
    del lexicon[word] #delete the words that are added to the list todelete from the list lexicon

c)Παρακάτω ανοίγω το ζητούμενο αρχείο και αποθηκεύω μέσα το λεξικό με τη μορφή που μας ζητήθηκε

In [ ]:
!mkdir vocab

vocab=open("vocab/words.vocab.txt","w")
for word in lexicon:
  vocab.write(word) #the word
  vocab.write("\t") #the tab between them
  vocab.write(str(lexicon[word])) #the value of the word
  vocab.write("\n") #change line
vocab.close()

mkdir: cannot create directory ‘vocab’: File exists


# Βημα 3:


3) a) Aντιστοιχίζουμε το γράμματα της αλφαβήτου με τα αντίστοιχα αυξανόμενα νούμερα αρχίζοντας από ε = 0

In [ ]:
def alphabet_indexing(alphabet):
	indexed_alphabet = []
	indexed_alphabet.append(("<epsilon>", 0)) #epsilon had value 0
	i = 1;
	for letter in alphabet:
		indexed_alphabet.append((letter, str(i))) #each letter has increasing value starting from a=1
		i+=1
	return indexed_alphabet

def symbols_file(ab, filename = "chars.syms"):
	f = open(filename, "w+")
	for pair in ab:
		f.write(pair[0] + "\t\t" + str(pair[1]) + "\n") #pair[0] is the word itself and pair[1] is its value
	
alphabet = ["a","b","c","d","e","f","g","h","i","j","k","l","m","n","o","p","q","r","s","t","u","v","w","x","y","z"]
symbols_file(alphabet_indexing(alphabet),"vocab/chars.syms")

b)Εδω γίνεται ομοίως η αντιστοίχιση για τις λέξεις του lexicon με αύξοντες αριθμούς

In [ ]:
def lexicon_indexing(lexicon):
	indexed_lexicon = []
	i = 0;
	for word in lexicon:
		indexed_lexicon.append((word, str(i)))
		i+=1
	return indexed_lexicon

symbols_file(lexicon_indexing(lexicon),"vocab/words.syms")

# Βημα 4:

4)a)Υλοποιούμε τον μετατροπέα

In [ ]:
def text_fst(Alphabet, filename = "text.txt"):
    f = open(filename, "w+")
    for pair in Alphabet:
        for pair1 in Alphabet:
            if  (pair[0] == pair1[0] =="<epsilon>"):
                continue 
            elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
            elif (pair[0]=='<epsilon>'):
                f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
            elif (pair1[0]=='<epsilon>'):
                f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
            else:
                f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 1"+ "\n")
    f.write("0")
    f.close()


β, γ)

In [ ]:
!mkdir fsts
text_fst(alphabet_indexing(alphabet),"fsts/L.fst")
!fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/L.fst > fsts/L.binfst
!fstinfo  fsts/L.binfst

mkdir: cannot create directory ‘fsts’: File exists
fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       1
# of arcs                                         728
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               26
# of output epsilons                              26
input label multiplicity                          26.9643
output label multiplicity                         26.9643
# of accessible states                            1
# of coaccessible states                          1
# of connected states                             1
# of connected components                         1
# of strongly conn components  

ζ)

In [ ]:
Alphabet = [('<epsilon>', 0), ('a', '1'), ('b', '2')]
symbols_file(Alphabet, "vocab/chars1.syms")
text_fst(Alphabet, "fsts/Lsmall.fst")

!fstcompile -isymbols=vocab/chars1.syms -osymbols=vocab/chars1.syms fsts/Lsmall.fst > fsts/Lsmall.binfst
!fstdraw --isymbols=vocab/chars1.syms --osymbols=vocab/chars1.syms -portrait fsts/Lsmall.binfst  | dot -Tjpg > fsts/Lsmall.jpg
!fstinfo fsts/Lsmall.binfst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       1
# of arcs                                         8
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               2
# of output epsilons                              2
input label multiplicity                          2.75
output label multiplicity                         2.75
# of accessible states                            1
# of coaccessible states                          1
# of connected states                             1
# of connected components                         1
# of strongly conn components                     1
input matcher                           

# Βήμα 5

5)α)Κατασκευάζω έναν αποδοχέα

In [ ]:
def Acceptor(Lexicon, Alphabet, filename = "V.fst"):
    for i in range(0, len(Alphabet)):
        temp = Alphabet[i]
        Alphabet[i] = [temp, i]
    f = open(filename, "w+")
    i = 2
    for word in Lexicon:
        for j in range(0, len(word)):
            if j==0:
                f.write("0" + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                # print("0" + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n") 
                i += 1
            elif (j== len(word) -1):
                f.write( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " 0" + "\n")
                # print( str(i-1) + " " + "1" + " " + word[j] + " " + word[j] + " 0" + "\n") 
                i +=1
            else:
                f.write( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                # print( str(i-1) + " " + str(i) + " " + word[j] + " " + word[j] + " 0" + "\n")
                i+=1
    f.write("1")
    # print("1")
    f.close()


β, γ, δ)

In [ ]:
Acceptor(lexicon,alphabet_indexing(alphabet),"fsts/V.fst")
!fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/V.fst > fsts/V.binfst
!fstinfo fsts/V.binfst
!fstdeterminize fsts/V.binfst  > fsts/V_determinized.binfst
!fstminimize fsts/V_determinized.binfst > fsts/V_minimized.binfst
!fstinfo fsts/V_minimized.binfst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       92745
# of arcs                                         108186
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          132.168
output label multiplicity                         132.168
# of accessible states                            92745
# of coaccessible states                          92722
# of connected states                             92722
# of connected components                         1
# of strongly conn components                     92745
input mat

ε)

In [ ]:
count = 0
newlexicon = {}
for i in lexicon:
  if count == 10:
    break
  newlexicon[i] = lexicon[i]
  count += 1
Acceptor(newlexicon,alphabet_indexing(alphabet),"fsts/Vsmall.fst")
!fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/Vsmall.fst > fsts/Vsmall.binfst
!fstinfo fsts/Vsmall.binfst
!fstdeterminize fsts/Vsmall.binfst  > fsts/Vsmall_determinized.binfst
!fstminimize fsts/Vsmall_determinized.binfst > fsts/Vsmall_minimized.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/Vsmall_minimized.binfst  | dot -Tjpg >fsts/Vsmall.jpg
!fstinfo fsts/Vsmall_minimized.binfst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       43
# of arcs                                         50
initial state                                     0
# of final states                                 1
# of input/output epsilons                        0
# of input epsilons                               0
# of output epsilons                              0
input label multiplicity                          1.04
output label multiplicity                         1.04
# of accessible states                            43
# of coaccessible states                          42
# of connected states                             42
# of connected components                         1
# of strongly conn components                     43
input matcher                     

In [ ]:
!fstcompose fsts/L.binfst fsts/V_minimized.binfst > fsts/Leven.binfst
!fstinfo fsts/Leven.binfst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       9141
# of arcs                                         746265
initial state                                     0
# of final states                                 1725
# of input/output epsilons                        0
# of input epsilons                               18837
# of output epsilons                              237666
input label multiplicity                          4.41912
output label multiplicity                         26.6815
# of accessible states                            9141
# of coaccessible states                          9141
# of connected states                             9141
# of connected components                         1
# of strongly conn components                     9141
in

# Βήμα 6

6.α, β)

In [ ]:
Lexicon_cit = ["cit"]
Acceptor(Lexicon_cit,alphabet_indexing(alphabet), "fsts/cit.fst")
! fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/cit.fst > fsts/cit.binfst
! fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/cit.binfst  | dot -Tjpg >fsts/cit.jpg
! fstcompose fsts/cit.binfst fsts/Leven.binfst  > fsts/min_distance1.binfst
! fstinfo fsts/min_distance1.binfst
!fstshortestpath fsts/min_distance1.binfst | fstrmepsilon > fsts/out1.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out1.binfst  | dot -Tjpg >fsts/out1.jpg
!fstinfo fsts/out1.binfst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       36564
# of arcs                                         159282
initial state                                     0
# of final states                                 1725
# of input/output epsilons                        0
# of input epsilons                               75348
# of output epsilons                              27423
input label multiplicity                          4.26835
output label multiplicity                         1.70957
# of accessible states                            36564
# of coaccessible states                          36564
# of connected states                             36564
# of connected components                         1
# of strongly conn components                     3656

In [ ]:
Lexicon_cwt = ["cwt"]
Acceptor(Lexicon_cwt,alphabet_indexing(alphabet), "fsts/cwt.fst")
! fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/cwt.fst > fsts/cwt.binfst
! fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/cwt.binfst  | dot -Tjpg >fsts/cwt.jpg
! fstcompose fsts/cwt.binfst fsts/Leven.binfst  > fsts/min_distance2.binfst
! fstinfo fsts/min_distance2.binfst
!fstshortestpath fsts/min_distance2.binfst | fstrmepsilon > fsts/out2.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out2.binfst  | dot -Tjpg >fsts/out2.jpg
!fstinfo fsts/out2.binfst

fst type                                          vector
arc type                                          standard
input symbol table                                none
output symbol table                               none
# of states                                       36564
# of arcs                                         159282
initial state                                     0
# of final states                                 1725
# of input/output epsilons                        0
# of input epsilons                               75348
# of output epsilons                              27423
input label multiplicity                          4.26835
output label multiplicity                         1.70957
# of accessible states                            36564
# of coaccessible states                          36564
# of connected states                             36564
# of connected components                         1
# of strongly conn components                     3656

# Βήμα 7


7)a)

In [ ]:
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download("https://raw.githubusercontent.com/slp-ntua/slp-labs/master/lab1/data/spell_test.txt","evaluation_data.txt")
print("All the files are downloaded")#If the downloaded file is a zip file than you can use below function to unzip it.

All the files are downloaded


b)

In [ ]:
!ls vocab/


chars1.syms  chars.syms  words.syms  words.vocab.txt


In [ ]:
import random
lines = []
with open("evaluation_data.txt") as file:
    for line in file:
        lines.append(line)
text = []
dictionary = {}
for line in lines:
    text = []
    text.append(tokenize(clean_text(line)))
    for i in range (1,len(text[0])):
      dictionary.update({text[0][i]:text[0][0]})
for counter in range(0,20):
  key = random.choice(list(dictionary.keys()))
  value = dictionary[key]
  Lexicon_test = [key]
  f = open('new.txt', 'w+')
  f.truncate(0)
  f.close()
  fileopen = open("new.txt","w+")
  for i in range(0,len(key)):
    fileopen.write("%s " %str(i))
    fileopen.write(" %s " %str(i+1))
    fileopen.write(" %s " %key[i])
    fileopen.write(" %s " %key[i])
    fileopen.write("0\n")
  fileopen.write(str(len(key)))
  fileopen.close()
  Acceptor(Lexicon_test, alphabet_indexing(alphabet), "new.txt")
  !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms new.txt > fsts/test.binfst
  !fstcompose fsts/test.binfst fsts/Leven.binfst  > fsts/min_distance.binfst
  print("Wrong word = " ,key,", actual word = ",value)
  print("What we got from the transducer =", end =" ")  
  !fstshortestpath fsts/min_distance.binfst | fstrmepsilon | fsttopsort | fstprint -osymbols=vocab/chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n'
  print()

Wrong word =  rember , actual word =  remember
What we got from the transducer = member
Wrong word =  sorces , actual word =  sources
What we got from the transducer = forces
Wrong word =  chaphter , actual word =  chapter
What we got from the transducer = chapter
Wrong word =  viseted , actual word =  visited
What we got from the transducer = visited
Wrong word =  buiscits , actual word =  biscuits
What we got from the transducer = biscuits
Wrong word =  dessicate , actual word =  desiccate
What we got from the transducer = delicate
Wrong word =  compair , actual word =  compare
What we got from the transducer = complain
Wrong word =  inconvienient , actual word =  inconvenient
What we got from the transducer = inconvenient
Wrong word =  proble , actual word =  problem
What we got from the transducer = problem
Wrong word =  somone , actual word =  someone
What we got from the transducer = soone
Wrong word =  bisquits , actual word =  biscuits
What we got from the transducer = biscuits

#Βημα 8

Tρέχοντας τις κατάλληλες εντολές στο Bash δημιουργούμε απο έναν αποδοχέα τόσο για τη σωστή όσο και για τη λάθος λέξη και εφ όσον η συνάρτηση μας Acceptor δέχεται στην είσοδο dictionary , εισάγουμε τις λέξεις ως τέτοια. 

In [ ]:
!mkdir nameit

LexiconAbandonned = {}
LexiconAbandonned["abandonned"] = ""
Acceptor(LexiconAbandonned,Alphabet,"fsts/Abandonned.txt")
LexiconAbandoned = {}
LexiconAbandoned["abandoned"] = ""
Acceptor(LexiconAbandoned,Alphabet,"fsts/Abandoned.txt")

!fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/Abandonned.txt > fsts/Abandonned.binfst
!fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/Abandoned.txt > fsts/Abandoned.binfst

!fstcompose fsts/Abandonned.binfst fsts/L.binfst > fsts/produced.binfst
!fstcompose fsts/produced.binfst fsts/Abandoned.binfst > fsts/MLN.fst

f = open("temporary.txt",mode = "w+")
f.close()
!fstshortestpath fsts/MLN.fst | fstprint --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms --show_weight_one > temporary.txt
with open("temporary.txt", "r") as a_file:
  content = a_file.readlines()
  for line in content:
    line_list = line.rstrip().split(sep = '\t') 
    print(line_list)
    # 

mkdir: cannot create directory ‘nameit’: File exists


δ)ε)Ουσιαστικά εδώ ανοίγω το αρχείο wiki αποθηκεύω τις γραμμές του σε μια μεταβλητή ( wiki_content) την οποία έπειτα τρέχω γραμμή γραμμή. Για κάθε στοιχείο της γραμμής (2 αφού υπάρχει μία η σωστή και μία η λάθος λέξη) δημιουργώ από ένα αυτόματο και φτιάχνω τη σύνθεση που κάνω και στα ερωτήματα α,β,γ της οποίας τον πίνακα μεταβάσεων αποθηκεύω κάθε φορά στο αρχείο temporary.Κάθε φορά τρέχω το αρχείο αυτό και κρατάω σε μια λίστα μόνο τις γραμμές του που αποτελούνται απο > 2 στοιχεία (ώστε να μην είναι τελική κατάσταση) και έχουν  line_list[4] != "0" (κόστος > 0) δηλαδή τις μεταβάσεις που διορθώνουν λάθος και στο αντίστοιχο key του dictionary τοποθετώ τον αριθμό των εμφανίσεων . Έτσι έχω και τα δύο ερωτήματα στο ίδιο dictionary .


In [ ]:
# SE SXOLIA GIATI PAIRNEI POLU WRA NA TREKSEI

# temporary_dictionary = {}
# with open("wiki.txt") as wiki: 
#     wiki_content=wiki.readlines()
#     for wiki_line in wiki_content:
#         wiki_list = wiki_line.rstrip().split(sep = '\t')
#         LexiconAbandonned = {}
#         LexiconAbandonned[wiki_list[0]] = ""
#         Acceptor(LexiconAbandonned,Alphabet,"fsts/Abandonned.txt")
#         LexiconAbandoned = {}
#         LexiconAbandoned[wiki_list[1]] = ""
#         Acceptor(LexiconAbandoned,Alphabet,"fsts/Abandoned.txt")
#         !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/Abandonned.txt > fsts/Abandonned.binfst
#         !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/Abandoned.txt > fsts/Abandoned.binfst
#         !fstcompose fsts/Abandonned.binfst fsts/L.binfst > fsts/produced.binfst
#         !fstcompose fsts/produced.binfst fsts/Abandoned.binfst > fsts/MLN.fst
#         f = open("temporary.txt",mode = "w+")
#         f.close()
#         !fstshortestpath fsts/MLN.fst | fstprint --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms --show_weight_one > temporary.txt
#         with open("temporary.txt", "r") as a_file:
#             content = a_file.readlines()
#             for line in content:
#                 line_list = line.rstrip().split(sep = '\t') 
#                 if len(line_list) > 2 and line_list[4] != "0":
#                     if (line_list[2],line_list[3]) not in temporary_dictionary:
#                         temporary_dictionary[(line_list[2],line_list[3])] = int(line_list[4])
#                     else:
#                         temporary_dictionary[(line_list[2],line_list[3])] += int(line_list[4])
# print(temporary_dictionary)

In [ ]:
f = open("edits.txt","w+")
for key, value in temporary_dictionary.items():
    f.write(str(key[0]) + "\t" + str(key[1]) + "\t" + str(value) + "\r")
f.close()

In [ ]:
import numpy as np
f = open("edits.txt","r")
temporary_dictionary = {}
content = f.readlines()
for line in content:
    line_list = line.rstrip().split(sep = '\t')
    temporary_dictionary[(line_list[0],line_list[1])] = int(line_list[2])
sum = 0
for x in temporary_dictionary:
    sum += int(temporary_dictionary[x])
for x in temporary_dictionary:
    temporary_dictionary[x] = - np.log2(temporary_dictionary[x] / sum)

print(temporary_dictionary)

{('n', '<epsilon>'): 6.149747119504682, ('<epsilon>', 'r'): 6.149747119504682, ('y', 'i'): 6.149747119504682, ('<epsilon>', 'i'): 4.564784618783526, ('o', 'a'): 4.149747119504682, ('b', '<epsilon>'): 6.149747119504682, ('u', 't'): 6.149747119504682, ('t', 'u'): 6.149747119504682, ('c', '<epsilon>'): 4.149747119504682, ('e', 'i'): 6.149747119504682, ('<epsilon>', 's'): 6.149747119504682, ('a', 'e'): 3.8278190246173196, ('i', 'a'): 4.564784618783526, ('<epsilon>', 'b'): 4.564784618783526, ('r', 'b'): 6.149747119504682, ('b', 'r'): 6.149747119504682, ('s', 'c'): 6.149747119504682, ('<epsilon>', 'e'): 5.149747119504682, ('s', 't'): 6.149747119504682, ('b', 'p'): 5.149747119504682, ('<epsilon>', 'n'): 6.149747119504682, ('i', 'c'): 6.149747119504682, ('c', 'n'): 6.149747119504682, ('i', '<epsilon>'): 6.149747119504682, ('u', 'a'): 6.149747119504682, ('t', '<epsilon>'): 6.149747119504682, ('l', '<epsilon>'): 6.149747119504682, ('n', 's'): 6.149747119504682, ('c', 's'): 6.149747119504682, ('e

στ)


In [ ]:
def edits_text_fst(Alphabet, temporary_dictionary, filename = "edits_text.txt"):
    f = open(filename, "w+")
    for pair in Alphabet:
        for pair1 in Alphabet:
            if  (pair[0] == pair1[0] =="<epsilon>"):
                continue 
            elif (pair[0] == pair1[0] and pair[0]!="<epsilon>"):
                f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " 0" + "\n")
            elif (pair[0]=='<epsilon>'):
                if (pair[0],pair1[0]) not in temporary_dictionary:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(1000)+ "\n")
                else:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(temporary_dictionary[(pair[0],pair1[0])])+ "\n")
            elif (pair1[0]=='<epsilon>'):
                if (pair[0],pair1[0]) not in temporary_dictionary:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(1000)+ "\n")
                else:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(temporary_dictionary[(pair[0],pair1[0])])+ "\n")            
            else:
                if (pair[0],pair1[0]) not in temporary_dictionary:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(1000)+ "\n")
                else:
                    f.write("0" + " 0 " + pair[0] + " " + pair1[0] + " " + str(temporary_dictionary[(pair[0],pair1[0])])+ "\n")    
    f.write("0")
    f.close()

edits_text_fst(alphabet_indexing(alphabet),temporary_dictionary,"fsts/edit_text.txt")
!fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/edit_text.txt > fsts/E.binfst


ζ)

In [ ]:
!fstcompose fsts/E.binfst fsts/V_minimized.binfst > fsts/New_Leven.binfst
# !fstinfo fsts/New_Leven.binfst
Lexicon_cit = ["cit"]
Acceptor(Lexicon_cit,alphabet_indexing(alphabet), "fsts/cit.fst")
! fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/cit.fst > fsts/cit.binfst
# ! fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/cit.binfst  | dot -Tjpg >fsts/cit.jpg
! fstcompose fsts/cit.binfst fsts/New_Leven.binfst  > fsts/min_distance1.binfst
# ! fstinfo fsts/min_distance1.binfst
!fstshortestpath fsts/min_distance1.binfst | fstrmepsilon > fsts/out3.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out3.binfst  | dot -Tjpg >fsts/out3.jpg
# !fstinfo fsts/out3.binfst

Lexicon_cwt = ["cwt"]
Acceptor(Lexicon_cwt,alphabet_indexing(alphabet), "fsts/cwt.fst")
! fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/cwt.fst > fsts/cwt.binfst
! fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/cwt.binfst  | dot -Tjpg >fsts/cwt.jpg
! fstcompose fsts/cwt.binfst fsts/New_Leven.binfst  > fsts/min_distance2.binfst
# ! fstinfo fsts/min_distance2.binfst
!fstshortestpath fsts/min_distance2.binfst | fstrmepsilon > fsts/out4.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out4.binfst  | dot -Tjpg >fsts/out4.jpg
# !fstinfo fsts/out4.binfst

import random
lines = []
with open("evaluation_data.txt") as file:
    for line in file:
        lines.append(line)
text = []
dictionary = {}
for line in lines:
    text = []
    text.append(tokenize(clean_text(line)))
    for i in range (1,len(text[0])):
      dictionary.update({text[0][i]:text[0][0]})
for counter in range(0,20):
  key = random.choice(list(dictionary.keys()))
  value = dictionary[key]
  Lexicon_test = [key]
  f = open('new.txt', 'w+')
  f.truncate(0)
  f.close()
  fileopen = open("new.txt","w+")
  for i in range(0,len(key)):
    fileopen.write("%s " %str(i))
    fileopen.write(" %s " %str(i+1))
    fileopen.write(" %s " %key[i])
    fileopen.write(" %s " %key[i])
    fileopen.write("0\n")
  fileopen.write(str(len(key)))
  fileopen.close()
  Acceptor(Lexicon_test, alphabet_indexing(alphabet), "new.txt")
  !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms new.txt > test.binfst
  !fstcompose test.binfst fsts/New_Leven.binfst  > min_distance.binfst
  print("Wrong word = " ,key,", actual word = ",value)
  print("What we got from the transducer =", end =" ")  
  !fstshortestpath min_distance.binfst | fstrmepsilon | fsttopsort | fstprint -osymbols=vocab/chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n'
  print()

# Βημα 9


α)

In [ ]:
import numpy as np

lexicon = {}
total_words = 0
for line in preprocessed:
  for word in line:
    if word in lexicon:
      lexicon[word] += 1 #if the word exist , increase by one its value
    else:
      lexicon[word] = 1 #if it doesn't exist initialize its value = 1
# print(word)
todelete = []
for word in lexicon:
  if lexicon[word] < 5:
    todelete.append(word) #if its value is less than 5 add it to a list named todelete
    # print(word)

for word in todelete:
    del lexicon[word] #delete the words that are added to the list todelete from the list lexicon

for word in lexicon:
    total_words += lexicon[word]

for word in lexicon:
    lexicon[word] = - np.log2(lexicon[word] / total_words)

β,γ,δ)

In [ ]:
def W_fst(Lexicon,Alphabet,filename = "fsts/W.fst"):
    f = open(filename, "w+")
    for word in Lexicon:
        f.write( "0" + " " + "0" + " " + word + " " + word + " " + str(lexicon[word]) + "\n")
    f.write("0")
    f.close()


# mylexicon = {}
# mylexicon["this"] = 3
# mylexicon["that"] = 5
W_fst(lexicon[0:3],Alphabet,)
!fstcompile -isymbols=vocab/words.syms -osymbols=vocab/words.syms fsts/W.fst > fsts/W.binfst

!fstcompose fsts/Leven.binfst fsts/W.binfst > fsts/LVW.binfst
!fstcompose fsts/New_Leven.binfst fsts/W.binfst > fsts/EVW.binfst

# !fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait  fsts/EVW.binfst  | dot -Tjpg > fsts/EVW.jpg
# !fstshortestpath fsts/EVW.binfst | fstrmepsilon > fsts/out7.binfst
# !fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out7.binfst  | dot -Tjpg >fsts/out7.jpg

ε,ζ)

In [ ]:

Lexicon_cit = ["cit"]
Acceptor(Lexicon_cit,alphabet_indexing(alphabet), "fsts/cit.fst")
! fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/cit.fst > fsts/cit.binfst
# ! fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/cit.binfst  | dot -Tjpg >fsts/cit.jpg
! fstcompose fsts/cit.binfst fsts/LVW.binfst  > fsts/min_distance1.binfst
# ! fstinfo fsts/min_distance1.binfst
!fstshortestpath fsts/min_distance1.binfst | fstrmepsilon > fsts/out5.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out5.binfst  | dot -Tjpg >fsts/out5.jpg
# !fstinfo fsts/out3.binfst

Lexicon_cwt = ["cwt"]
Acceptor(Lexicon_cwt,alphabet_indexing(alphabet), "fsts/cwt.fst")
! fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms fsts/cwt.fst > fsts/cwt.binfst
! fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/cwt.binfst  | dot -Tjpg >fsts/cwt.jpg
! fstcompose fsts/cwt.binfst fsts/LVW.binfst  > fsts/min_distance2.binfst
# ! fstinfo fsts/min_distance2.binfst
!fstshortestpath fsts/min_distance2.binfst | fstrmepsilon > fsts/out6.binfst
!fstdraw --isymbols=vocab/chars.syms --osymbols=vocab/chars.syms -portrait fsts/out6.binfst  | dot -Tjpg >fsts/out6.jpg
# !fstinfo fsts/out4.binfst

import random
lines = []
with open("evaluation_data.txt") as file:
    for line in file:
        lines.append(line)
text = []
dictionary = {}
for line in lines:
    text = []
    text.append(tokenize(clean_text(line)))
    for i in range (1,len(text[0])):
      dictionary.update({text[0][i]:text[0][0]})
for counter in range(0,5):
  key = random.choice(list(dictionary.keys()))
  value = dictionary[key]
  Lexicon_test = [key]
  f = open('new.txt', 'w+')
  f.truncate(0)
  f.close()
  fileopen = open("new.txt","w+")
  for i in range(0,len(key)):
    fileopen.write("%s " %str(i))
    fileopen.write(" %s " %str(i+1))
    fileopen.write(" %s " %key[i])
    fileopen.write(" %s " %key[i])
    fileopen.write("0\n")
  fileopen.write(str(len(key)))
  fileopen.close()
  Acceptor(Lexicon_test, alphabet_indexing(alphabet), "new.txt")
  !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms new.txt > test.binfst
  !fstcompose test.binfst fsts/LVW.binfst  > min_distance.binfst
  print("Wrong word = " ,key,", actual word = ",value)
  print("What we got from the transducer =", end =" ")  
  !fstshortestpath min_distance.binfst | fstrmepsilon | fsttopsort | fstprint -osymbols=vocab/chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n'
  print()


Wrong word =  initals , actual word =  initials
What we got from the transducer = ii
Wrong word =  muinets , actual word =  minutes
What we got from the transducer = vi
Wrong word =  ment , actual word =  meant
What we got from the transducer = vi
Wrong word =  poartry , actual word =  poetry
What we got from the transducer = vi
Wrong word =  occurence , actual word =  occurrence
What we got from the transducer = ur


# Βημα 10


In [ ]:
def Leven_accuracy(fst,filename):
    with open(filename,"r") as f:
        lines = f.readlines()
        correct = 0
        total = 0
        for line in lines:
            line = line.replace(":","")
            line = line.split()
            for i in range(len(line) - 1):
                f = open('new.txt', 'w+')
                f.truncate(0)
                f.close()
                fileopen = open("new.txt","w+")
                for j in range(len(line[i+1])):
                    fileopen.write("%s " %str(j))
                    fileopen.write(" %s " %str(j+1))
                    fileopen.write(" %s " %line[i+1][j])
                    fileopen.write(" %s " %line[i+1][j])
                    fileopen.write("0\n")
                fileopen.write(str(len(line[i+1])))

                fileopen.close()
                # break
                Lexicon_test = {}
                Lexicon_test[line[i+1]] = line[i+1]
                Acceptor(Lexicon_test, alphabet_indexing(alphabet), "new.txt")
                !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms new.txt > test.binfst
                !fstcompose test.binfst fsts/Leven.binfst  > min_distance.binfst
                f = open('new.txt', 'w+')
                f.truncate(0)
                f.close()
                
                !fstshortestpath min_distance.binfst | fstrmepsilon | fsttopsort | fstprint -osymbols=vocab/chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n' > new.txt
                fileopen = open("new.txt","r")
                for content in fileopen.readlines():
                    content = content.rstrip().split(sep = '\t')
                    if content[0] == line[0]:
                        correct += 1
                    total += 1
    print("LV Accuracy =",round(correct/total,3))

def New_Leven_accuracy(fst,filename):
    with open(filename,"r") as f:
        lines = f.readlines()
        correct = 0
        total = 0
        for line in lines:
            line = line.replace(":","")
            line = line.split()
            for i in range(len(line) - 1):
                f = open('new.txt', 'w+')
                f.truncate(0)
                f.close()
                fileopen = open("new.txt","w+")
                for j in range(len(line[i+1])):
                    fileopen.write("%s " %str(j))
                    fileopen.write(" %s " %str(j+1))
                    fileopen.write(" %s " %line[i+1][j])
                    fileopen.write(" %s " %line[i+1][j])
                    fileopen.write("0\n")
                fileopen.write(str(len(line[i+1])))

                fileopen.close()
                # break
                Lexicon_test = {}
                Lexicon_test[line[i+1]] = line[i+1]
                Acceptor(Lexicon_test, alphabet_indexing(alphabet), "new.txt")
                !fstcompile -isymbols=vocab/chars.syms -osymbols=vocab/chars.syms new.txt > test.binfst
                !fstcompose test.binfst fsts/New_Leven.binfst  > min_distance.binfst
                f = open('new.txt', 'w+')
                f.truncate(0)
                f.close()
                
                !fstshortestpath min_distance.binfst | fstrmepsilon | fsttopsort | fstprint -osymbols=vocab/chars.syms | cut -f4 | grep -v "<epsilon>" | head -n -1 | tr -d '\n' > new.txt
                fileopen = open("new.txt","r")
                for content in fileopen.readlines():
                    content = content.rstrip().split(sep = '\t')
                    if content[0] == line[0]:
                        correct += 1
                    total += 1
    print("EV Accuracy =",round(correct/total,3))


In [ ]:
Leven_accuracy(_,"spell_test.txt")

In [ ]:
New_Leven_accuracy(_,"spell_test.txt")

# Βημα 12


In [ ]:
import logging
import multiprocessing
import os

from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

CORPUS = "gutenberg" 
raw_corpus = download_corpus(corpus=CORPUS)
preprocessed = process_file(raw_corpus, preprocess=preprocess)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
# Enable gensim logging
logging.basicConfig(
    format="%(levelname)s - %(asctime)s: %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO,
)


class W2VLossLogger(CallbackAny2Vec):
    """Callback to print loss after each epoch
    use by passing model.train(..., callbacks=[W2VLossLogger()])
    """

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()

        if self.epoch == 0:
            print("Loss after epoch {}: {}".format(self.epoch, loss))
        else:
            print(
                "Loss after epoch {}: {}".format(
                    self.epoch, loss - self.loss_previous_step
                )
            )
        self.epoch += 1
        self.loss_previous_step = loss


def train_w2v_model(
    sentences,
    output_file,
    window=5,
    embedding_dim=100,
    epochs=300,
    min_word_count=10,
):
    """Train a word2vec model based on given sentences.
    Args:
        sentences list[list[str]]: List of sentences. Each element contains a list with the words
            in the current sentence
        output_file (str): Path to save the trained w2v model
        window (int): w2v context size
        embedding_dim (int): w2v vector dimension
        epochs (int): How many epochs should the training run
        min_word_count (int): Ignore words that appear less than min_word_count times
    """
    # raise NotImplementedError("You should use gensim to train your w2v model")
    workers = multiprocessing.cpu_count()

    # TODO: Instantiate gensim.models.Word2Vec class
    model =  Word2Vec(size = embedding_dim, window = window, min_count = min_word_count)
    # TODO: Build model vocabulary using sentences    
    model.build_vocab(sentences, progress_per=10000)
    # TODO: Train word2vec model
    model.train(sentences=sentences, epochs = epochs, total_examples=model.corpus_count, total_words=model.corpus_total_words, callbacks=[W2VLossLogger()])
    # model.train(..., callbacks=[W2VLossLogger()])
    # Save trained model
    model.save(output_file)

    return model

CORPUS = 'gutemberg'
sentences = preprocessed
output_file = "gutenberg_w2v.100d.model"
window = 5
embedding_dim = 100
epochs = 1000
min_word_count = 5

train_w2v_model(
    sentences,
    output_file,
    window=window,
    embedding_dim=embedding_dim,
    epochs=epochs,
    min_word_count=min_word_count,
)


In [ ]:
#min to peiraxeis grapse apo panw tha sou pw ti einai ayto 
!zip -r /content/vocab.zip /content/vocab
!zip -r /content/data.zip /content/data
!zip -r /content/fsts.zip /content/fsts

from google.colab import files
files.download("/content/vocab.zip","/content/data.zip","/content/fsts.zip")

In [ ]:
model = Word2Vec.load("gutenberg_w2v.100d.model")
prediction1 = {}
for word in ['bible', 'book', 'bank', 'water']:
  prediction1[word] = model.wv.most_similar(word, topn=1)
  print(prediction1[word])

In [ ]:
prediction2 = {}
for word in [('girls', 'queen', 'kings'), ('good', 'tall', 'taller'), ('france', 'paris', 'london')]:
  prediction2[word] = model.wv.most_similar(positive=[word[0], word[2]], negative=[word[1]], topn=1)
  print(prediction2[word])

In [ ]:
!wget  -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit= 1000000)

In [ ]:
prediction1 = {}
for word in ['bible', 'book', 'bank', 'water']:
  prediction1[word] = model.wv.most_similar(word, topn=1)
  print(prediction1[word])

In [ ]:
prediction2 = {}
for word in [('girls', 'queen', 'kings'), ('good', 'tall', 'taller'), ('france', 'paris', 'london')]:
  prediction2[word] = model.wv.most_similar(positive=[word[0], word[2]], negative=[word[1]], topn=1)
  print(prediction2[word])

# Βημα 13

In [ ]:
!cd drive/'My Drive'/
embedding_file = open("embeddings.tsv","w")
metadata_file = open("metadata.tsv","w")
model = Word2Vec.load("gutenberg_w2v.100d.model")
model.wv["bible"]
for word in lexicon:
    for dim in range(99):
        embedding_file.write(str(model.wv[word][dim]))
        embedding_file.write("\t")
    embedding_file.write(str(model.wv[word][99]))
    embedding_file.write("\n")
    metadata_file.write(word)
    metadata_file.write("\n")
embedding_file.close()
metadata_file.close()

INFO - 18:26:00: loading Word2Vec object from gutenberg_w2v.100d.model
INFO - 18:26:00: loading wv recursively from gutenberg_w2v.100d.model.wv.* with mmap=None
INFO - 18:26:00: setting ignored attribute vectors_norm to None
INFO - 18:26:00: loading vocabulary recursively from gutenberg_w2v.100d.model.vocabulary.* with mmap=None
INFO - 18:26:00: loading trainables recursively from gutenberg_w2v.100d.model.trainables.* with mmap=None
INFO - 18:26:00: setting ignored attribute cum_table to None
INFO - 18:26:00: loaded gutenberg_w2v.100d.model


In [ ]:
from google.colab import files

files.download('embeddings.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Βημα 14


In [ ]:
!wget  -c "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

--2020-11-16 13:55:24--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  22.3MB/s    in 5.3s    

2020-11-16 13:55:29 (15.0 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!tar -xf aclImdb_v1.tar.gz 

In [ ]:
import glob
import os
import re

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import sklearn


data_dir = './aclImdb/'
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
pos_train_dir = os.path.join(train_dir, "pos")
neg_train_dir = os.path.join(train_dir, "neg")
pos_test_dir = os.path.join(test_dir, "pos")
neg_test_dir = os.path.join(test_dir, "neg")

# For memory limitations. These parameters fit in 8GB of RAM.
# If you have 16G of RAM you can experiment with the full dataset / W2V
MAX_NUM_SAMPLES = 5000
# Load first 1M word embeddings. This works because GoogleNews are roughly
# sorted from most frequent to least frequent.
# It may yield much worse results for other embeddings corpora
NUM_W2V_TO_LOAD = 1000000


SEED = 42

# Fix numpy random seed for reproducibility
np.random.seed(SEED)


def strip_punctuation(s):
    return re.sub(r"[^a-zA-Z\s]", " ", s)


def preprocess(s):
    return re.sub("\s+", " ", strip_punctuation(s).lower())


def tokenize(s):
    return s.split(" ")


def preproc_tok(s):
    return tokenize(preprocess(s))


def read_samples(folder, preprocess=lambda x: x):
    samples = glob.iglob(os.path.join(folder, "*.txt"))
    data = []

    for i, sample in enumerate(samples):
        if MAX_NUM_SAMPLES > 0 and i == MAX_NUM_SAMPLES:
            break
        with open(sample, "r") as fd:
            x = [preprocess(l) for l in fd][0]
            data.append(x)

    return data


def create_corpus(pos, neg):
    corpus = np.array(pos + neg)
    y = np.array([1 for _ in pos] + [0 for _ in neg])
    indices = np.arange(y.shape[0])
    np.random.shuffle(indices)

    return list(corpus[indices]), list(y[indices])


def extract_nbow(corpus,model,dimentions):
    """Extract neural bag of words representations"""
    nbow = np.zeros((len(corpus),dimentions))
    for i in range(len(corpus)):
        for j in range(len(corpus[i])):
            if corpus[i][j] in model.wv.vocab:
                nbow[i] +=  model.wv[corpus[i][j]]
        nbow[i] = nbow[i]/ len(corpus[i])
    return nbow
    # raise NotImplementedError("Implement nbow extractor")


def train_sentiment_analysis(train_corpus, train_labels):
    """Train a sentiment analysis classifier using NBOW + Logistic regression"""
    logisticRegr = LogisticRegression()
    logisticRegr.fit(train_corpus, train_labels)
    return logisticRegr
    # raise NotImplementedError("Implement sentiment analysis training")


def evaluate_sentiment_analysis(classifier, test_corpus, test_labels):
    """Evaluate classifier in the test corpus and report accuracy"""
    y_pred = classifier.predict(test_corpus)
    correct = 0
    for i in range(len(y_pred)):
        if int(y_pred[i]) == test_labels[i]:
            correct += 1
    return round(correct/len(y_pred),3)


positive_train_data = read_samples(pos_train_dir)
negative_train_data = read_samples(neg_train_dir)
positive_test_data = read_samples(pos_test_dir)
negative_test_data = read_samples(neg_test_dir)
whole_positive = positive_train_data + positive_test_data
whole_negative = negative_train_data + negative_test_data

train_corpus,train_labels = create_corpus(whole_positive[0:8000],whole_negative[0:8000])
test_corpus,test_labels = create_corpus(whole_positive[8000:9999],whole_negative[8000:9999])

In [ ]:
model = Word2Vec.load("gutenberg_w2v.100d.model")
train_nbow = extract_nbow(train_corpus,model,100)
test_nbow = extract_nbow(test_corpus,model,100)
Reg = train_sentiment_analysis(train_nbow,train_labels)
evaluate_sentiment_analysis(Reg,test_nbow,test_labels)

Our model had accuracy = 0.612


In [ ]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit= 1000000)
train_nbow = extract_nbow(train_corpus,model,300)
test_nbow = extract_nbow(test_corpus,model,300)
Reg = train_sentiment_analysis(train_nbow,train_labels)
evaluate_sentiment_analysis(Reg,test_nbow,test_labels)

Google's model had accuracy = 0.602
